In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd


In [ ]:
hashtag = "#SupremeCourt"
num_tweets = 20


In [ ]:
tweets_list = []


In [ ]:
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(hashtag + ' lang:en').get_items()):
    if i >= num_tweets:
        break
    tweets_list.append([tweet.id, tweet.conversationId, tweet.date, tweet.user.id, tweet.user.username, 
                        tweet.user.displayname, tweet.place, tweet.rawContent, tweet.lang, tweet.mentionedUsers,
                       tweet.links, tweet.media, tweet.replyCount, tweet.retweetCount, 
                       tweet.likeCount, tweet.hashtags, tweet.cashtags, tweet.source, tweet.retweetedTweet, 
                       tweet.quotedTweet, tweet.inReplyToUser,tweet.inReplyToTweetId, tweet.viewCount])


In [ ]:
tweets_df = pd.DataFrame(tweets_list, columns=['id', 'conversation_id', 'date','user_id', 'username', 
                                               'name', 'place', 'tweet', 'language', 'mentions',
                                               'urls', 'photos', 'replies_count', 'retweets_count', 
                                               'likes_count','hashtags', 'cashtags', 'source', 'retweet', 
                                               'quote_url','reply_to', 'reply_to_id','view_count'])


In [ ]:
tweets_df.to_csv('custom_twitter_trend_dataset.csv', index=False)


In [73]:
df2 = pd.read_csv('./custom_twitter_trend_dataset.csv')
# df2['photos']
# total_tweets = df2['photos'].sum()
# print(total_tweets)

non_media_tweets = df2['photos'].isnull().sum()
media_tweets = num_tweets-non_media_tweets

max_likedtweets_indexes = df2.nlargest(num_tweets, 'likes_count')[
    'likes_count'].index.tolist()

max_liked_tweets = df2.loc[max_likedtweets_indexes, 'tweet'].tolist()

max_retweets_count_indexes = df2.nlargest(num_tweets, 'retweets_count')[
    'likes_count'].index.tolist()

max_retweets_tweets = df2.loc[max_retweets_count_indexes, 'tweet'].tolist()

df2['media_tweets'] = media_tweets
df2['text_tweets'] = non_media_tweets

df2['number_max_liked_tweets'] = max_likedtweets_indexes
df2['max_liked_tweets'] = max_liked_tweets

df2['number_max_retweets_tweets'] = max_retweets_count_indexes
df2['max_retweets_tweets'] = max_retweets_tweets

check_bot_human = 'bot' # 0 - 1

df2['check_bot_human'] = check_bot_human

df2.columns




Index(['id', 'conversation_id', 'date', 'user_id', 'username', 'name', 'place',
       'tweet', 'language', 'mentions', 'urls', 'photos', 'replies_count',
       'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'source',
       'retweet', 'quote_url', 'reply_to', 'reply_to_id', 'view_count',
       'media_tweets', 'text_tweets', 'number_max_liked_tweets',
       'max_liked_tweets', 'number_max_retweets_tweets', 'max_retweets_tweets',
       'check_bot_human'],
      dtype='object')

In [74]:
selected_columns = ['username', 'check_bot_human']
df2[selected_columns].to_csv('test.csv', index=False)


In [93]:
df = pd.read_csv('./tweets_24_notendencias_raw.csv', nrows=1)
print(df.columns)


Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'conversation_id', 'created_at',
       'date', 'time', 'timezone', 'user_id', 'username', 'name', 'place',
       'tweet', 'language', 'mentions', 'urls', 'photos', 'replies_count',
       'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'link',
       'retweet', 'quote_url', 'video', 'thumbnail', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')


In [94]:
df = pd.read_csv('./custom_twitter_trend_dataset.csv')
print(df.columns)


Index(['id', 'conversation_id', 'date', 'user_id', 'username', 'name', 'place',
       'tweet', 'language', 'mentions', 'urls', 'photos', 'replies_count',
       'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'source',
       'retweet', 'quote_url', 'reply_to', 'reply_to_id', 'view_count'],
      dtype='object')


In [ ]:
df = pd.read_csv('./tweets_24_notendencias_raw.csv', nrows=1)
df.head()


In [ ]:
df = pd.read_csv('./custom_twitter_trend_dataset.csv')
df.head()


In [70]:
json_object = df2.to_json()

print(json_object)


{"id":{"0":1641098280523243520,"1":1641098241352560641,"2":1641098229608529920,"3":1641098161191002113,"4":1641097897935511553,"5":1641097800229199878,"6":1641097724530683904,"7":1641097706478133248,"8":1641097613368778753,"9":1641097540786348043,"10":1641097381495345152,"11":1641097243045289986,"12":1641097219104284674,"13":1641097180424413184,"14":1641097171758923778,"15":1641096844661997572,"16":1641096679133675523,"17":1641096410652082176,"18":1641096187221532673,"19":1641095627751735302},"conversation_id":{"0":1641098280523243520,"1":1641098241352560641,"2":1641098229608529920,"3":1641098161191002113,"4":1641097897935511553,"5":1641097800229199878,"6":1641097724530683904,"7":1641097706478133248,"8":1641097613368778753,"9":1641097540786348043,"10":1641097381495345152,"11":1641097243045289986,"12":1641097219104284674,"13":1641097180424413184,"14":1641097171758923778,"15":1641096844661997572,"16":1641096679133675523,"17":1641096410652082176,"18":1641096187221532673,"19":1641095627751

In [71]:
import json
from pathlib import Path

# jsonobjc = json.dumps(json_object)
# data_folder = Path("")
# file_to_open = data_folder / "jsonobjtrend.json"
# file = open(file_to_open, "w")
# file.write(jsonobjc)
# file.close()

if (json_object):
    # Writing to json
    data_folder = Path("")
    file_to_open = data_folder / "jsonobjtrend.json"
    file = open(file_to_open, "w")
    file.write(json_object)
    file.close()


In [91]:
# checking tweet made by bot accounts
count=0
for i in range(num_tweets):
    username = df2.loc[i, 'username']
    value = df2.loc[i, 'check_bot_human']

    print(f"{username, value}")

df3 = pd.read_csv("./test.csv")

# value = (df2.iloc[i]['username'])
# if (df2['username'].isin([value]).any()):
#     count1 = count1 + 1


('princess_ara__', 'bot')
('Realsamosky', 'bot')
('montero4emirate', 'bot')
('ObioOluebube', 'bot')
('ptifamily', 'bot')
('DreyfusLawFirm', 'bot')
('OjhaaRajesh', 'bot')
('qas_hayat', 'bot')
('BlufCrazycomic', 'bot')
('DrHamtab', 'bot')
('MirrorNow', 'bot')
('Standard_ngr', 'bot')
('skybabym', 'bot')
('apricot5_it', 'bot')
('zahir_jafri', 'bot')
('PiNewsUpdates', 'bot')
('Clermont1', 'bot')
('KenDicci1', 'bot')
('ChocnewsC', 'bot')
('BlufCrazycomic', 'bot')


In [113]:

# checking df2 made by bot accounts
count = 0
for i in range(num_tweets):
   value = (df2['username'][i])
   if (df3['username'].isin([value]).any()):
       count = count+1

print('no of tweet made by bot overall', count)

# filter dataframe by time
df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d %H:%M:%S%z')
df2 = df2.sort_values(by='date')


# access time filtered datafram index by index
for i in range(1):
    print(df2.iloc[i]['username'])
    print(df2.iloc[i]['date'])

# checkig ratio of bot in first 1000
count1 = 0
for i in range(18):
   value = (df2.iloc[i]['username'])
   if (df3['username'].isin([value]).any()):
       count1 = count1 + 1

print('bot number of df2 in first 1000 tweets', count1)


#







# finding df2 in first 2 hour
print('start time is ')
hcount = 0
start_time = pd.to_datetime(df2.iloc[0]['date'], format='%H:%M:%S')
print('start time is ', start_time)

end_time = start_time + pd.Timedelta(hours=2)
print('end time is ', end_time)


hcount = len(df2[(df2['date'] >= start_time) & (df2['date'] < end_time)])
print('df2 in first 2 hours is :', hcount)

# # alternative finding df2 in first 2 hour
# for i in range (3000):
#     if(df2.iloc[i]['date']< end_time):
#         hcount = hcount+1


# print('df2 in first 2 hours is :',hcount);


# unique account participation
unique_account = df2['username'].nunique()
print("Unique account participation is :", unique_account)

# unique message in trend
unique_tweet = df2['tweet'].nunique()
print('unique df2 is :', unique_tweet)

# finding velocity of trend

print('')
print('Finding Acceleration.')
start_time = pd.to_datetime(df2.iloc[0]['date'], format='%Y-%m-%d %H:%M:%S%z')
print('start time is ', start_time)
i = 1
acceleration_list = []
hour_list = []
count_list = []
merge_list = []
for i in range(6):

    end_time = start_time + pd.Timedelta(hours=1)
    print('end time is ', end_time)

    hcount = len(df2[(df2['date'] >= start_time) & (df2['date'] < end_time)])
    # Create a boolean mask for df2s in the time range
    time_mask = (df2['date'] >= start_time) & (df2['date'] < end_time)

    # Select the user IDs of df2s in the time range
    users_in_range = df2.loc[time_mask, 'username']

    # Count the number of bot users in the time range
    num_bot_users = users_in_range.isin(df3['username']).sum()
    print('num of bot user ', num_bot_users)
    print("df2 in first " + str(i+1) + " hours is :", hcount)
    count_list.append(hcount)
    hour_list.append(i+1)
    if (i == 0):
        acceleration_list.append(hcount)
    else:
        acceleration_list.append(hcount - acceleration_list[i-1])
    start_time = end_time

# print(acceleration_list)
print(count_list)


no of tweet made by bot overall 20
BlufCrazycomic
2023-03-29 15:11:02+00:00
bot number of df2 in first 1000 tweets 18
start time is 
start time is  2023-03-29 15:11:02+00:00
end time is  2023-03-29 17:11:02+00:00
df2 in first 2 hours is : 20
Unique account participation is : 19
unique df2 is : 20

Finding Acceleration.
start time is  2023-03-29 15:11:02+00:00
end time is  2023-03-29 16:11:02+00:00
num of bot user  20
df2 in first 1 hours is : 20
end time is  2023-03-29 17:11:02+00:00
num of bot user  0
df2 in first 2 hours is : 0
end time is  2023-03-29 18:11:02+00:00
num of bot user  0
df2 in first 3 hours is : 0
end time is  2023-03-29 19:11:02+00:00
num of bot user  0
df2 in first 4 hours is : 0
end time is  2023-03-29 20:11:02+00:00
num of bot user  0
df2 in first 5 hours is : 0
end time is  2023-03-29 21:11:02+00:00
num of bot user  0
df2 in first 6 hours is : 0
[20, 0, 0, 0, 0, 0]


In [ ]:
# unique accounts
# unique messages
# bot list by hour
# human list by hour
# ok.